In [ ]:
import os

# 1. 查看当前占用显卡的进程 (如果有 rocm-smi)
print("🔍 检查当前显卡占用情况:")
!rocm-smi --showpids

# 2. 强制杀掉所有 Python 进程 (核弹级清理)
# 注意：这会重启当前的 Notebook 内核，你需要重新运行前面的 import 和 config 代码
print("💣 正在清理所有 Python 进程...")
!pkill -9 python
!pkill -9 python3

print("✅ 清理完成。请等待 5 秒后，点击 Notebook 上方的 'Restart Kernel'，然后从头运行配置。")

🔍 检查当前显卡占用情况:


============================ ROCm System Management Interface ============================
===================================== KFD Processes ======================================
KFD process information:
PID 	PROCESS NAME	GPU(s)	VRAM USED	SDMA USED	CU OCCUPANCY	
3021	UNKNOWN     	0     	0        	0        	0           	
================================== End of ROCm SMI Log ===================================


In [1]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
print("🧹 显存清理完毕")

🧹 显存清理完毕


In [ ]:
import os

# ✅ 修正：使用和你截图中完全一致的文件夹名称
folder_name = "LlamaFactory" 

# 检查当前位置
print(f"当前工作目录: {os.getcwd()}")

if os.path.exists(folder_name):
    os.chdir("/workspace/LlamaFactory")
    print(f"📂 已切换工作目录至: {os.getcwd()}")
    
    # 切换进去后再运行安装命令
    print("🚀 开始安装依赖...")
    !pip install -e ".[metrics]"
    !pip install deepspeed
    print("✅ 环境依赖准备就绪")
else:
    print(f"❌ 依然找不到目录 '{folder_name}'")
    # 打印一下当前目录都有啥，方便调试
    print(f"当前目录下的文件: {os.listdir('.')}")

In [ ]:
import json

# 你的数据文件名 (请确保该文件在 /workspace 下，或者写绝对路径)
# 根据你的截图，数据文件应该在上一级目录
DATA_FILE_PATH = "/workspace/evol_instruct_dataset.json" 
DATASET_NAME = "my_evol_data"

# 读取原本的 dataset_info.json
info_file = "data/dataset_info.json"
with open(info_file, "r", encoding="utf-8") as f:
    dataset_info = json.load(f)

# 添加你的数据集配置
dataset_info[DATASET_NAME] = {
    "file_name": DATA_FILE_PATH,
    "columns": {
        "prompt": "instruction",  # 对应 Distilabel 生成的 instruction
        "response": "response",   # 对应 Distilabel 生成的 response
        # "system": "system"      # 如果你的数据里没有 system 列，这行可以不写
    }
}

# 写回文件
with open(info_file, "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, indent=2, ensure_ascii=False)

print(f"🎉 数据集 [{DATASET_NAME}] 已注册成功！")

In [8]:
import os
import yaml
import torch
import gc

# ================= 🧹 1. 再次确信显存是空的 =================
print("🧹 正在执行最后的显存检查与清理...")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    # 打印当前显存占用，确保是真的空了 (应该在 1GB 以内)
    print(f"📉 当前显存已分配: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"📉 当前显存保留中: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print("✅ 准备就绪")

# 确保在正确的工作目录
if os.path.exists("LlamaFactory"):
    os.chdir("LlamaFactory")
print(f"📂 当前工作目录: {os.getcwd()}")

# ================= ⚙️ 2. 生成 32GB 满血版配置 =================
config = {
    # --- 模型与数据 ---
    "model_name_or_path": "/data/models/Qwen2.5-7B-Instruct",
    "dataset": "my_evol_data",
    "template": "qwen",
    "overwrite_cache": True,
    "preprocessing_num_workers": 16,

    # --- 输出配置 ---
    "output_dir": "/workspace/saves/qwen2.5-lora-32g-full",
    "logging_steps": 5,          # 正常步数记录
    "save_steps": 50,            # 每 50 步存档
    "plot_loss": True,
    "overwrite_output_dir": True,

    # --- 训练核心 ---
    "stage": "sft",
    "do_train": True,
    "finetuning_type": "lora",
    "lora_target": "all",
    
    # 🔥 性能升级点 A: Rank 翻倍，更聪明的微调
    "lora_rank": 32,
    "lora_alpha": 32,
    "lora_dropout": 0.05,

    # --- 硬件优化 (32GB 满血模式) ---
    
    # 🔥 性能升级点 B: 解锁 4k 长上下文
    "cutoff_len": 4096,
    
    # 🔥 性能升级点 C: Batch Size 翻 4 倍 (吞吐量大增)
    "per_device_train_batch_size": 4,
    
    # 🔥 配合调整: 4(BS) * 4(Accum) = 16 (总有效 Batch)
    "gradient_accumulation_steps": 4,
    
    "learning_rate": 1.0e-4,
    "num_train_epochs": 10.0,    # 设为 10 轮，确保即使数据少也能跑一会儿
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    
    # ⚡ MI50 核心铁律: 必须 FP16，严禁 BF16
    "fp16": True,
    
    "ddp_timeout": 18000000,
    # 只有2条数据时建议关闭验证，防止报错；数据多时可开启
    "val_size": 0.0, 
    # "eval_strategy": "steps",
    # "eval_steps": 100,
    # "per_device_eval_batch_size": 4,
}

# 保存配置文件
config_file = "train_config_mi50_32g_full.yaml"
with open(config_file, "w", encoding="utf-8") as f:
    yaml.dump(config, f, sort_keys=False)

print(f"📝 满血版配置文件已生成: {config_file}")

# ================= 🚀 3. 开始训练 =================
print("🚀 全速启动训练 (Performance Mode)...")
print("   (观察显存: 如果长时间不动请不要慌，正在分配大块显存)")

!FORCE_TORCHRUN=1 llamafactory-cli train train_config_mi50_32g_full.yaml

🧹 正在执行最后的显存检查与清理...
📉 当前显存已分配: 0.00 GB
📉 当前显存保留中: 0.00 GB
✅ 准备就绪
📂 当前工作目录: /workspace/LlamaFactory
📝 满血版配置文件已生成: train_config_mi50_32g_full.yaml
🚀 全速启动训练 (Performance Mode)...
   (观察显存: 如果长时间不动请不要慌，正在分配大块显存)
[INFO|2026-01-12 09:53:29] llamafactory.launcher:143 >> Initializing 1 distributed tasks at: 127.0.0.1:45739
[WARNING|2026-01-12 09:53:42] llamafactory.extras.misc:154 >> Version checking has been disabled, may lead to unexpected behaviors.
[W112 09:53:46.472984849 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())
[INFO|2026-01-12 09:53:46] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
[INFO|2026-01-12 09:53:46] llamafactory.hparams.parser:465 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: True, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2093] 2026-01-12 09:53:46,20